In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch

from rbm.definitions import DATASET_DIR, OUTPUT_DIR
from rbm.models import TMCRBM2D

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
dtype = torch.float

# Train a TMC-RBM on the 2d3c artificial dataset using the TMC2D algorithm

In [2]:
data = torch.tensor(np.genfromtxt(DATASET_DIR.joinpath('data_2d_3c_balanced_seed12.d')).T, dtype=dtype, device=device)
# The data is binary with values in {-1, 1}, so we scale it to be in {0, 1}
data = (data+1)/2

The cell below defines all the parameters necessary to initialize the TMC-RBM

In [3]:
# Learning rate
lr = 0.2
# Number of MCMC iterations
NGibbs = 20
# Mini-batch size
mb_s = 2000
# Number of hidden units 
Nh = 50 
# Number of visible units
Nv = data.shape[0]
# steps used for the temporal mean
it_mean = 15
# constraint on the gaussian bath
N = 20000
# nb of chains for each constraint point
nb_chain = 50
# nb of constraint point on each dimension
nb_point_dim = torch.tensor([30,30])
# if True uses PCA for the projection else uses the SVD of the weight matrix
PCA = True
# index of the direction you want to put the constraints on (careful : first direction is 0, second is 1 etc...)
direction = torch.tensor([0, 1], device = device)
# the distance added to the extremas of the data projection for the discretization
border_length = 0.2
# permanent chain
ResetChain = False
# Wether to use Centered gradient update or not
UpdCentered = True
# Number of persistent Markov chains for the PCD algorithm. Not useful in this notebook but should be initialized if 
# one wants to use the PCD method alongside the TMC
num_pcd = 200

In [4]:
# We initialize the model using all the previous parameters
RBM_TMC = TMCRBM2D(num_visible=Nv,
               num_hidden=Nh,
               device=device,
               lr=lr,
               gibbs_steps=NGibbs,
               UpdCentered=True,
               CDLearning=True,
               mb_s=mb_s,
               num_pcd=num_pcd,
               N = N,
               nb_chain = nb_chain,
               nb_point_dim=nb_point_dim,
               direction=direction,
               PCA = PCA,
               border_length = border_length)

In [5]:
# Number of training epochs
ep_max = 1020

# All saved filenames will have this stamp 
RBM_TMC.file_stamp = 'Demo2d3cB'

# We define here the timesteps where we will save the RBM during training
fq_msr_RBM = 20
base = 1.7
v = np.array([0,1],dtype=int)
allm = np.append(np.array(0),base**np.array(list(range(30))))
for k in range(30):
    for m in allm:
        v = np.append(v,int(base**k)+int(m)) 
v = np.array(list(set(v)))
v = np.sort(v)
RBM_TMC.list_save_time = v
RBM_TMC.list_save_rbm = np.arange(1, ep_max, fq_msr_RBM)

# Visible bias initialisation
RBM_TMC.SetVisBias(data)

In [6]:
# Epoch frequency on which to generate figures in the output folders
# Useful to monitor the learning
fq_fig = 20

if RBM_TMC.PCA:
    _, _, V = torch.svd(data.T)
    proj_data = torch.mm(data.T,V).cpu()/ (RBM_TMC.Nv**0.5)


for t in range(ep_max):
    RBM_TMC.fit(data,ep_max=1)
    
    if t%fq_fig == 0:
        if not(RBM_TMC.PCA):
            _,_,V = torch.svd(RBM_TMC.W)
            proj_data = torch.mm(data.T,V).cpu()/ (RBM_TMC.Nv**0.5)
        
        fig = plt.figure()
        plt.scatter(proj_data[:, RBM_TMC.direction[0]], proj_data[:,RBM_TMC.direction[1]], label = "dataset")
        cont1 = plt.contour(RBM_TMC.w_hat_tmp[0], RBM_TMC.w_hat_tmp[1], np.log(RBM_TMC.p_m.cpu().numpy()), 50)
        plt.title("RBM Potential")
        plt.colorbar(cont1)
        plt.legend()
        plt.savefig(OUTPUT_DIR.joinpath(f"{RBM_TMC.file_stamp}_fig_pot_{t}.png"))
        plt.close(fig)
        
        fig = plt.figure()
        plt.scatter(proj_data[:, RBM_TMC.direction[0]], proj_data[:,RBM_TMC.direction[1]], label = "dataset")
        cont2 = plt.contour(RBM_TMC.w_hat_tmp[0], RBM_TMC.w_hat_tmp[1], RBM_TMC.p_m.cpu().numpy(), 50)
        plt.title('RBM Probability')
        plt.legend()
        plt.colorbar(cont2)
        plt.savefig(OUTPUT_DIR.joinpath(f"{RBM_TMC.file_stamp}_fig_proba_{t}.png"))
        plt.close(fig)

IT  0


/home/nbereux/rbm/rbm/models/tmcrbm2d.py:421: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/torch/csrc/utils/tensor_new.cpp:210.)
  self.w_hat_b = torch.tensor(


Saving nb_upd=0
Saving nb_upd=1
Saving nb_upd=2
Saving nb_upd=3
Saving nb_upd=4
model updates saved at /home/nbereux/rbm/model/AllParametersDemo2d3cB.h5
model saved at /home/nbereux/rbm/model/RBMDemo2d3cB.h5


/tmp/ipykernel_33960/1302182158.py:20: RuntimeWarning: divide by zero encountered in log
  cont1 = plt.contour(RBM_TMC.w_hat_tmp[0], RBM_TMC.w_hat_tmp[1], np.log(RBM_TMC.p_m.cpu().numpy()), 50)


IT  1
Saving nb_upd=5
Saving nb_upd=6
Saving nb_upd=8
Saving nb_upd=9
model updates saved at /home/nbereux/rbm/model/AllParametersDemo2d3cB.h5
model saved at /home/nbereux/rbm/model/RBMDemo2d3cB.h5
IT  2
Saving nb_upd=10
Saving nb_upd=12
Saving nb_upd=14
model updates saved at /home/nbereux/rbm/model/AllParametersDemo2d3cB.h5
model saved at /home/nbereux/rbm/model/RBMDemo2d3cB.h5
IT  3
Saving nb_upd=15
Saving nb_upd=16
Saving nb_upd=18
model updates saved at /home/nbereux/rbm/model/AllParametersDemo2d3cB.h5
model saved at /home/nbereux/rbm/model/RBMDemo2d3cB.h5
IT  4
Saving nb_upd=22
Saving nb_upd=24
model updates saved at /home/nbereux/rbm/model/AllParametersDemo2d3cB.h5
model saved at /home/nbereux/rbm/model/RBMDemo2d3cB.h5
IT  5
Saving nb_upd=25
Saving nb_upd=26
Saving nb_upd=28
model updates saved at /home/nbereux/rbm/model/AllParametersDemo2d3cB.h5
model saved at /home/nbereux/rbm/model/RBMDemo2d3cB.h5
IT  6
Saving nb_upd=32
model updates saved at /home/nbereux/rbm/model/AllParame